<a href="https://colab.research.google.com/github/1nc0gn30/1nc0gn30/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install TensorFlow documentation tools
!pip install -q git+https://github.com/tensorflow/docs

# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

# Check TensorFlow version
try:
    %tensorflow_version 2.x
except Exception:
    pass


In [ ]:
# Download the dataset
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv

# Load into Pandas DataFrame
dataset = pd.read_csv('insurance.csv')

# Display last few rows of the dataset
dataset.tail()


In [ ]:
# Convert categorical data to numerical
dataset['sex'] = dataset['sex'].map({'male': 0, 'female': 1})
dataset['smoker'] = dataset['smoker'].map({'yes': 1, 'no': 0})
dataset['region'] = dataset['region'].astype('category').cat.codes  # Convert categorical region to numeric

# Shuffle the dataset
dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate features and labels
train_size = int(len(dataset) * 0.8)

train_dataset = dataset.iloc[:train_size].copy()
test_dataset = dataset.iloc[train_size:].copy()

# Pop off the labels (expenses column)
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')


In [ ]:
# Define the model
model = keras.Sequential([
    normalizer,  # Use the normalized input
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Output layer with a single neuron for regression
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),  # Use Adam optimizer
              loss='mse',  # Mean Squared Error loss
              metrics=['mae', 'mse'])  # Track Mean Absolute Error (MAE) and MSE

# Print model summary to verify
model.summary()


In [ ]:
# Train the model
history = model.fit(
    train_features,  # Use NumPy array instead of Pandas DataFrame
    train_labels,
    validation_split=0.2,
    epochs=200,
    verbose=1  # Set to 1 to see training progress
)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
